<a href="https://colab.research.google.com/github/BehnoushSh/LLMs/blob/main/Scocial_Media_Sentiment_Analysis_Open_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flipside import Flipside
import numpy as np
import pandas as pd
import openai
from dune_client.client import DuneClient
from dune_client.query import QueryBase
from dune_client.models import (
    ExecutionResponse,
    ExecutionStatusResponse,
    ResultsResponse,
    ExecutionResultCSV,
    DuneError,
)

In [ ]:
import config

sdkf = Flipside(config.sdkf)
openai.api_key = config.API_KEY_openai
dune_api_key = config.API_KEY_dune

below is Query to fetch all token names and nft collection names on ethereum ecosystem.

In [ ]:
query1 = """select DISTINCT symbol, name
from crosschain.price.dim_asset_metadata
where blockchain = 'ethereum'
"""

data = sdkf.query(query1)

In [ ]:
result_list = data.records
result_df = pd.DataFrame(result_list)
result_df

,symbol,name,__row_index
0,HOGE,hoge.finance,0
1,BOBO,Bobo,1
2,zUSD,Zerogoki USD,2
3,BPVC,BORED PEPE VIP CLUB,3
4,HOT,HoloToken,4
...,...,...,...
12814,AORA,AORA COIN,12814
12815,SAUDISHIB,SAUDI SHIBA INU,12815
12816,FFYI,FiscusFYI,12816
12817,RST,Raini Studios Token,12817


Here I tried fetching the casts using Dune api, but their api is broken at the moment. For the product itself we might not need dune api if we go directly to Neynar api.

In [ ]:
# dune_query = QueryBase(
#     name="vahid test",
#     query_id=3635087,
# )


# dune = DuneClient(
#     api_key= dune_api_key,  # Dune API key
#     base_url='https://api.dune.com',
#     request_timeout=10
# )
# results_df = dune.run_query_dataframe(query = dune_query)



Finally, I downloaded a all casts in the month of april from Neynar in a csv.

In [ ]:
casts = pd.read_csv('All casts april.csv')
casts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134542 entries, 0 to 134541
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   fid         134542 non-null  int64 
 1   text        130178 non-null  object
 2   created_at  134542 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.1+ MB


This is the infamous for loop. lol. be careful running it.
zero values mean that that token/collection had no casts on farcaster in April 2024.
---



In [ ]:
token_counts = {}

for token in result_df['name']:
    # Initialize the count of the current token to 0
    token_counts[token] = 0
    # Iterate through each row in dataframe B to count occurrences of the token
    for text in casts['text']:
        token_counts[token] += str(text).split().count(token)

# Output the total count of each token
token_counts

{'hoge.finance': 0,
 'Bobo': 0,
 'Zerogoki USD': 0,
 'BORED PEPE VIP CLUB': 0,
 'HoloToken': 0,
 'CBK': 0,
 'Amazing Unit': 0,
 'Granny Shiba': 0,
 'SportX': 0,
 'Linkart': 0,
 'https://unimex.network/': 0,
 'IlliquidDAO': 0,
 'Instadapp WBTC': 0,
 'ElectrifyAsia': 0,
 'Bitrock': 0,
 'Promodio': 0,
 'AllStars Coin': 0,
 'Eiichiro Oda': 0,
 'BarnBridge Governance Token': 0,
 'Equus': 0,
 'FRENCH ICO COIN': 0,
 'VELASPAD.io': 0,
 'Distributed Energy Coin': 0,
 'Purple AI': 0,
 'Museum of Crypto Art': 0,
 'MeetToken': 0,
 'Eco Value Coin': 0,
 'Wrapped AVAX (Wormhole)': 0,
 'hiBAYC': 0,
 'Pitch FXS': 0,
 'PEPE FLOKI': 0,
 None: 0,
 'Meowl': 0,
 'Camp': 5,
 'Tacos': 0,
 'ChainTools': 0,
 'Nimbus': 0,
 'Gemini dollar': 0,
 'Defiking': 0,
 'FaithCoin': 0,
 'Wrapped BEAM': 0,
 'Shido Inu': 0,
 'Blockpass': 0,
 'Catex': 0,
 '0xNumber': 0,
 'BNPL Pay': 0,
 'EANTO Token': 0,
 'KEEPSCOIN': 0,
 'IOU Loyalty Exchange Token': 0,
 'Ethersocks': 0,
 'Base Protocol': 0,
 'Sonic': 2,
 'Piggy Bank Token'

Here, I am looking at the casts to eliminate the false positives.
False positive are casts that we have detected to include the name of the token. But the cast is talking about a different thing and the similarity is accidental.
For instance, a cast might say "monarchs of persian gulf". it includes the word Monarch which is an nft collection but we don't include this cast in our sentiment analysis.

In [ ]:
def is_crypto_related(text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Is the following text related to cryptocurrencies or NFTs? only answer 'Yes' or 'No'."}
            ]
        )
        # Adjust the following line according to how the response structure is defined for chat completions
        answer = response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        answer = "Error"
    return answer

In [ ]:
casts['is_crypto_related'] = casts['text'].apply(is_crypto_related)


In [ ]:
casts_yes = casts[casts['is_crypto_related']== 'Yes']

Here we don the sentiment analysis

In [ ]:
def sentiment(text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"what is the sentiment of the writer behind this tweet? answer 'positive', 'negative' or 'neutral' with their respective probability: {text}"}
            ]
        )
        # Adjust the following line according to how the response structure is defined for chat completions
        answer = response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        answer = "Error"
    return answer

In [ ]:
casts_yes['sentiment'] = casts_yes['text'].apply(sentiment)